# 위치정보요약 데이터의 좌표 정보로 각 도로명 주소별 평균 위도, 경도 table 만들기

In [75]:
# pyproj로 주소정보 데이터 위도, 경도로 바꿔보기

import pyproj
from pyproj import Proj
from pyproj import transform

In [76]:
#연세대학교 위치. X과표 : 동서, Y좌표 : 남북, GRS80 UTM-K
# +proj=tmerc, +lat_0=38, +lon_0=127.5, +k=0.9996, +x_0=1000000, +y_0=2000000, +ellps=GRS80, +units=m, +no_defs

lx=950254.198668
ly=1951390.210053

# 구글 지도 epsg 3857 or epsg 900913
# +proj=merc, +a=6378137, +b=6378137, +lat_ts=0.0, +lon_0=0.0, +x_0=0.0, +y_0=0, +k=1.0, +units=m, +nadgrids=@null, +no_defs ->이건 구글 map 좌표.
# 우리가 필요한 건 경도, 위도
# https://www.osgeo.kr/17

# transfom(p1, p2, x,y, z=None, radians =False) : p1에서 정의된 x1,y1,z1 점을 p2에서 정의된 x2,y2로 바꾸는 것. z, radian은 default가 none,false인 듯 

In [77]:
GRS80={'proj':'tmerc', 'lat_0':'38', 'lon_0':'127.5', 'k':0.9996, 'x_0':1000000, 'y_0':2000000, 'ellps':'GRS80', 'units':'m',}
p1=Proj(**GRS80)

In [78]:
#EPSG={'proj':'merc', 'a':6378137, 'b':6378137, 'lat_ts':'0.0', 'lon_0':'0.0', 'x_0':0.0, 'y_0':0, 'k':1.0, 'units':'m'}
p2=Proj(init='epsg:4019')

In [103]:
nlx,nly = transform(p1,p2,lx,ly)

In [80]:
map_pos=list((nly,nlx))
#m=folium.Map(location=map_pos,min_zoom=11, max_zoom=20)

map_pos

[37.56052492190205, 126.93674890563082]

In [82]:
import pandas as pd
import numpy as np
import os
import glob #파일 한꺼번에 부르기 위해서 사용
import re

In [85]:
glob.glob(os.path.join("gh-data/loc-data","*.*"))

['gh-data/loc-data\\entrc_busan.txt',
 'gh-data/loc-data\\entrc_busan.xlsx',
 'gh-data/loc-data\\entrc_chungbuk.txt',
 'gh-data/loc-data\\entrc_chungbuk.xlsx',
 'gh-data/loc-data\\entrc_chungnam.txt',
 'gh-data/loc-data\\entrc_chungnam.xlsx',
 'gh-data/loc-data\\entrc_daegu.txt',
 'gh-data/loc-data\\entrc_daegu.xlsx',
 'gh-data/loc-data\\entrc_daejeon.txt',
 'gh-data/loc-data\\entrc_daejeon.xlsx',
 'gh-data/loc-data\\entrc_gangwon.txt',
 'gh-data/loc-data\\entrc_gangwon.xlsx',
 'gh-data/loc-data\\entrc_gwangju.txt',
 'gh-data/loc-data\\entrc_gwangju.xlsx',
 'gh-data/loc-data\\entrc_gyeongbuk.txt',
 'gh-data/loc-data\\entrc_gyeongbuk.xlsx',
 'gh-data/loc-data\\entrc_gyeongnam.txt',
 'gh-data/loc-data\\entrc_gyeongnam.xlsx',
 'gh-data/loc-data\\entrc_gyunggi.txt',
 'gh-data/loc-data\\entrc_incheon.txt',
 'gh-data/loc-data\\entrc_jeju.txt',
 'gh-data/loc-data\\entrc_jeonbuk.txt',
 'gh-data/loc-data\\entrc_jeonnam.txt',
 'gh-data/loc-data\\entrc_sejong.txt',
 'gh-data/loc-data\\entrc_seoul

In [86]:
filenames = glob.glob(os.path.join("gh-data/loc-data","entrc*.xlsx"))
filenames

['gh-data/loc-data\\entrc_busan.xlsx',
 'gh-data/loc-data\\entrc_chungbuk.xlsx',
 'gh-data/loc-data\\entrc_chungnam.xlsx',
 'gh-data/loc-data\\entrc_daegu.xlsx',
 'gh-data/loc-data\\entrc_daejeon.xlsx',
 'gh-data/loc-data\\entrc_gangwon.xlsx',
 'gh-data/loc-data\\entrc_gwangju.xlsx',
 'gh-data/loc-data\\entrc_gyeongbuk.xlsx',
 'gh-data/loc-data\\entrc_gyeongnam.xlsx']

In [87]:
df0=pd.read_excel('gh-data/loc-data/entrc_busan.xlsx',header = 0,encoding='utf-8')

In [88]:
df0.head()

,26110,2608,2611010100,부산광역시,중구,영주동,261103006001,초량상로,0,1,4,Unnamed: 11,48910,근린생활시설,0.1,영주제1동,1139873.245417,1680775.057503
0,26110,4754,2611010100,부산광역시,중구,영주동,261103006001,초량상로,0,5,0,NaN,48910,유통시설,0,영주제1동,1.139887e+06,1.680802e+06
1,26110,9293,2611010100,부산광역시,중구,영주동,261103006001,초량상로,0,6,0,NaN,48911,종교시설,0,영주제1동,1.139904e+06,1.680799e+06
2,26110,3054,2611010100,부산광역시,중구,영주동,261103006001,초량상로,0,9,0,NaN,48906,근린생활시설,0,영주제1동,1.139915e+06,1.680846e+06
3,26110,7440,2611010100,부산광역시,중구,영주동,261103006001,초량상로,0,9,2,NaN,48906,주택,0,영주제1동,1.139907e+06,1.680841e+06
4,26110,6910,2611010100,부산광역시,중구,영주동,261103006001,초량상로,0,9,6,NaN,48906,주택,0,영주제1동,1.139903e+06,1.680832e+06


In [89]:
df=pd.read_excel('gh-data/entrc_seoul_head.xlsx',header = 0,encoding='utf-8')

In [91]:
df_refine=df[['시도','시군구','도로명','x좌표','y좌표']]

In [97]:
df_refine.dtypes

시도      object
시군구     object
도로명     object
x좌표    float64
y좌표    float64
dtype: object

In [98]:
df_refine.keys()

Index(['시도', '시군구', '도로명', 'x좌표', 'y좌표'], dtype='object')

In [106]:
df_refine['x좌표'][0], df_refine['y좌표'][0]

(953241.683263, 1954023.466812)

In [108]:
a,b= transform(p1,p2,df_refine['x좌표'][0], df_refine['y좌표'][0])

a,b

(126.97040554796257, 37.58441543603041)

In [112]:
len(df_refine)

558393

In [119]:
df_refine.describe()

,x좌표,y좌표,위도,경도
count,557953.000000,5.579530e+05,558393.000000,558393.000000
mean,954889.003853,1.950512e+06,279196.000000,279196.000000
std,7176.593866,5.700535e+03,161194.318769,161194.318769
min,935361.053341,1.936900e+06,0.000000,0.000000
25%,948941.637493,1.945504e+06,139598.000000,139598.000000
50%,955721.448904,1.950530e+06,279196.000000,279196.000000
75%,960388.144223,1.954620e+06,418794.000000,418794.000000
max,971977.523101,1.966768e+06,558392.000000,558392.000000


In [167]:
lx=df_refine['x좌표'][67]
ly=df_refine['y좌표'][67]

lx>0

False

In [173]:
gx=[]
gy=[]
null=[]

#
#67번 데이터,

for k in list(range(len(df_refine))) :
    lx=df_refine['x좌표'][k]
    ly=df_refine['y좌표'][k]
    if lx>0 and ly>0 :
        a,b= transform(p1,p2,lx,ly)
        gx.append(a)
        gy.append(b)
    else :
        gx.append(0)
        gy.append(0)
        null.append(k)


len(null)

440

In [185]:
df_refine['위도']=gy
df_refine['경도']=gx

df_refine

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,시도,시군구,도로명,x좌표,y좌표,위도,경도
0,서울특별시,종로구,자하문로,953241.683263,1.954023e+06,37.584415,126.970406
1,서울특별시,종로구,자하문로,953243.258151,1.954034e+06,37.584513,126.970423
2,서울특별시,종로구,자하문로,953240.877646,1.954055e+06,37.584697,126.970394
3,서울특별시,종로구,자하문로,953195.459364,1.954079e+06,37.584916,126.969878
4,서울특별시,종로구,자하문로,953193.346779,1.954097e+06,37.585080,126.969853
5,서울특별시,종로구,자하문로,953236.255907,1.954065e+06,37.584793,126.970341
6,서울특별시,종로구,자하문로,953245.729952,1.954084e+06,37.584959,126.970448
7,서울특별시,종로구,자하문로,953206.487048,1.954092e+06,37.585033,126.970003
8,서울특별시,종로구,자하문로,953205.690304,1.954106e+06,37.585159,126.969993
9,서울특별시,종로구,자하문로,953239.617210,1.954115e+06,37.585241,126.970376


In [186]:
type(null)

list

In [187]:
df_refine_final=df_refine.drop(null,0) #가로일때 0, 세로일때 1

In [188]:
df_refine_final.describe()

,x좌표,y좌표,위도,경도
count,557953.000000,5.579530e+05,557953.000000,557953.000000
mean,954889.003853,1.950512e+06,37.552817,126.989294
std,7176.593866,5.700535e+03,0.051483,0.081139
min,935361.053341,1.936900e+06,37.430483,126.768162
25%,948941.637493,1.945504e+06,37.507623,126.921938
50%,955721.448904,1.950530e+06,37.552956,126.998685
75%,960388.144223,1.954620e+06,37.589997,127.051426
max,971977.523101,1.966768e+06,37.699493,127.182773


In [193]:
a1=df_refine_final['시도'][0]
a2=df_refine_final['시군구'][0]
a3=df_refine_final['도로명'][0]
ab=[a1,a2,a3]
new=' '.join(ab) #이렇게 하는게 처리 속도가 더 빠르다고 함

new

'서울특별시 종로구 자하문로'

In [203]:
df_refine_final.reset_index(inplace=True,drop=True)

address=[]

#list(range(len(df_refine_final)))

for i in list(range(len(df_refine_final))) :
    a1=df_refine_final['시도'][i]
    a2=df_refine_final['시군구'][i]
    a3=df_refine_final['도로명'][i]
    ab=[a1,a2,a3]
    new=' '.join(ab)
    address.append(new)

address

['서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 자하문로',
 '서울특별시 종로구 

In [204]:
df_refine_final["도로주소"]=address

AveSeoul=pd.pivot_table(df_refine_final, index=["도로주소"])

In [205]:
AveSeoul

,x좌표,y좌표,경도,위도
도로주소,,,,
서울특별시 강남구 강남대로,958087.110153,1.944950e+06,127.025802,37.502866
서울특별시 강남구 강남대로100길,958293.189028,1.944846e+06,127.028140,37.501941
서울특별시 강남구 강남대로102길,958251.261594,1.944949e+06,127.027660,37.502868
서울특별시 강남구 강남대로106길,958186.231392,1.944984e+06,127.026922,37.503174
서울특별시 강남구 강남대로110길,958313.842824,1.945043e+06,127.028362,37.503714
서울특별시 강남구 강남대로112길,958172.200834,1.945286e+06,127.026746,37.505896
서울특별시 강남구 강남대로114길,958154.381415,1.945332e+06,127.026542,37.506311
서울특별시 강남구 강남대로116길,958105.112677,1.945361e+06,127.025983,37.506573
서울특별시 강남구 강남대로118길,958162.120746,1.945415e+06,127.026624,37.507064


In [208]:
AveSeoul.to_excel('gh-data/2018_loc_for_add_in_seoul.xlsx',encoding='utf-8')

In [209]:
AA=pd.read_excel('gh-data/2018_loc_for_add_in_seoul.xlsx',index_col=None,encoding='utf-8')

AA

,도로주소,x좌표,y좌표,경도,위도
0,서울특별시 강남구 강남대로,958087.110153,1.944950e+06,127.025802,37.502866
1,서울특별시 강남구 강남대로100길,958293.189028,1.944846e+06,127.028140,37.501941
2,서울특별시 강남구 강남대로102길,958251.261594,1.944949e+06,127.027660,37.502868
3,서울특별시 강남구 강남대로106길,958186.231392,1.944984e+06,127.026922,37.503174
4,서울특별시 강남구 강남대로110길,958313.842824,1.945043e+06,127.028362,37.503714
5,서울특별시 강남구 강남대로112길,958172.200834,1.945286e+06,127.026746,37.505896
6,서울특별시 강남구 강남대로114길,958154.381415,1.945332e+06,127.026542,37.506311
7,서울특별시 강남구 강남대로116길,958105.112677,1.945361e+06,127.025983,37.506573
8,서울특별시 강남구 강남대로118길,958162.120746,1.945415e+06,127.026624,37.507064
9,서울특별시 강남구 강남대로120길,958184.775782,1.945466e+06,127.026878,37.507526
